# Анализ норматива оборотных средств предприятия

## Цель работы

Определение потребности предприятия в оборотных средствах и анализ показателей использования оборотных средств, отражающих скорость оборачиваемости этих средств в процессе производства и реализации продукции. Для этого необходимо:

1. Рассчитать норматив оборотных средств.
2. Определить величину высвобождения (вовлечения) денежных средств в результате ускорения (замедления) оборачиваемости оборотных средств.

Под нормативом (лимитом) оборотных средств понимается сумма оборотных средств, обеспечивающих планомерную и бесперебойную работу предприятия. Нормативы разрабатываются по каждому элементу нормируемых оборотных средств с учетом конкретных условий работы, специфики производства, условий снабжения и сбыта, периодичности расходования материальных ресурсов, длительности производственного цикла.

## Исходные данные (вариант 10)

### Общая характеристика оборотных средств предприятия

In [ ]:
import pandas as pd

table1 = pd.DataFrame(data={
    'Единица измерения': ['руб.', 'руб.', 'руб.', 'дн.', 'дн.', 'руб.', 'дн.', 'руб.', 'дн.'],
    'Показатель': [817000, 272000, 37022, 28, 4, 22500, 3, 1062100, 4]
}, index=[
    'Выпуск продукции по себестоимости',
    'Расход сырья, основных материалов, покупных изделий и полуфабрикатов',
    'Норматив по прочим оборотным средствам в составе производственных запасов',
    'Интервал между поставками',
    'Длительность цикла изготовления продукции',
    'Норматив оборотных средств по элементу "Расходы будущих периодов"',
    'Норма запаса готовой продукции',
    'Объем реализованной продукции в оптовых ценах',
    'Увеличение (уменьшение) длительности одного оборота'
])
table1

## Расчет норматива оборотных средств выполняется по следующей схеме:

### 1. Норматив оборотных средств по элементу "Производственные запасы"

Включает в себя:
* норматив по сырью, основным материалам, покупным изделиям и полуфабрикатам;
* норматив по прочим оборотным средствам в составе производственных запасов (вспомогательные материалы, топливо, запасные части, МБП) ― задается в исходных данных.

**Cреднесуточный расход** определяется по следующей формуле:

$$\overline{Р}_{сут} = \frac{Р_{пл}}{Д_{пл}}$$

где:

* $Р_{пл}$ — расход ресурса в плановом периоде, 
* $Д_{пл}$ — число календарных дней в плановом периоде (90 дней)

In [ ]:
import numpy as np
avg_daily_cost = table1['Показатель']['Расход сырья, основных материалов, покупных изделий и полуфабрикатов'] / 90
f'{np.round(avg_daily_cost, 2)} руб'

**Норма запаса в днях** $Д_н$ определяется как:

$$Д_н = J + Д_н^{тр} + Д_н^{стр}$$

где:
* $J$ — интервал между поставками (определяет норму текущего запаса)
* $Д_н^{тр}$ — норма транспортного запаса
* $Д_н^{стр}$ — норма страхового запаса

In [ ]:
daily_stock = table1['Показатель']['Интервал между поставками'] + (table1['Показатель']['Интервал между поставками'] / 2) + (table1['Показатель']['Интервал между поставками'] / 4)
f'{int(daily_stock)} дн'

**Норматив оборотных средств в производственных запасах** определяется по формуле:

$$Н_о^{пз} = \overline{Р}_{сут} \cdot Д_н$$

In [ ]:
supplies_asset_quota = avg_daily_cost * daily_stock
f'{np.round(supplies_asset_quota, 2)} руб'

### 2. Норматив оборотных средств по незавершенному производству

**Плановая себестоимость среднесуточного выпуска продукции** определяется как:

$$\overline{C}_{сут} = \frac{С_{пл}}{Д_{пл}}$$

где $С_{пл}$ — выпуск продукции по себестоимости в плановом периоде.

In [ ]:
prime_cost_day = table1['Показатель']['Выпуск продукции по себестоимости'] / 90
f'{np.round(prime_cost_day, 2)} руб/дн'

$$Н_о^{нп} = \overline{C}_{сут} \cdot Т_ц \cdot К_{нз}$$

где $К_{нз}$ — коэффициент нарастания затрат (принимается равным 0.5)

In [ ]:
prime_cost_quota = prime_cost_day * table1['Показатель']['Длительность цикла изготовления продукции'] * 0.5
f'{np.round(prime_cost_quota, 2)} руб'

### 3. Норматив оборотных средств по элементу "Расходы будущих периодов"

Задается в исходных данных

### 4. Норматив оборотных средств по элементу "Готовая продукция"

$$Н_о^{гп} = \overline{C}_{сут} \cdot Д_н^{гп}$$

где $Д_н^{гп}$ — норма запаса годовой продукции в днях.

In [ ]:
prod_asset_quota = prime_cost_day * table1['Показатель']['Норма запаса готовой продукции']
f'{np.round(prod_asset_quota, 2)} руб'

### 5. Общий норматив оборотных средств

$$Н_о = Н_о^{пз} + Н_о^{нп} + Н_о^{рб} + Н_о^{гп}$$

In [ ]:
asset_quota = supplies_asset_quota + prime_cost_quota + table1['Показатель']['Норматив оборотных средств по элементу "Расходы будущих периодов"'] + prod_asset_quota
f'{np.round(asset_quota, 2)} руб'

### 6. Оборачиваемость оборотных средств (количество оборотов) в плановом периоде

$$К_{об} = \frac{Q_{рп}}{Н_о}$$

где $Q_{рп}$ — объем реализованной продукции в оптовых ценах.

In [ ]:
asset_turnover_terms = table1['Показатель']['Объем реализованной продукции в оптовых ценах'] / asset_quota
f'{np.round(asset_turnover_terms, 2)}'

### 7. Длительность одного оборота

$$Т_{об} = \frac{Д_{пл}}{К_{об}}$$

In [ ]:
term_duration = 90 / asset_turnover_terms
f'{np.round(term_duration, 2)}'

### 8. Количество оборотов при ускорении (замедлении) оборачиваемости оборотных средств

$$К' = \frac{Д_{пл}}{Т_{об} \pm Т}$$

In [ ]:
term_count_prime = 90 / (term_duration + table1['Показатель']['Увеличение (уменьшение) длительности одного оборота'])
f'{np.round(term_count_prime, 2)}'

### 9. Норматив оборотных средств при ускорении (замедлении) оборачиваемости оборотных средств

$$Н'_о = \frac{Q_{рп}}{К'}$$

In [ ]:
asset_quota_prime = table1['Показатель']['Объем реализованной продукции в оптовых ценах'] / term_duration
f'{np.round(asset_quota_prime, 2)} руб.'

### 10. Величина высвобождения (или дополнительного вовлечения) денежных средств из оборота в результате ускорения (замедления) оборачиваемости оборотных средств

$$\pm \Delta Н_о = Н_о - Н'_о$$

In [ ]:
delta_asset_quota = asset_quota - asset_quota_prime
f'{np.round(asset_quota_prime, 2)} руб.'